<a href="https://colab.research.google.com/github/Dimasilham7/Line-bot-php/blob/master/Jantung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libs  
# NOTE: Run this one code, then restart this runtime and run again for next all... (PENTING!!!) 

### please update Albumentations to version>=0.3.0 for `Lambda` transform support
!pip install -U --pre segmentation-models --user

Requirement already up-to-date: segmentation-models in /root/.local/lib/python3.6/site-packages (1.0.1)


In [2]:
!pip show segmentation_models


Name: segmentation-models
Version: 1.0.1
Summary: Image segmentation models with pre-trained backbones with Keras.
Home-page: https://github.com/qubvel/segmentation_models
Author: Pavel Yakubovskiy
Author-email: qubvel@gmail.com
License: MIT
Location: /root/.local/lib/python3.6/site-packages
Requires: efficientnet, image-classifiers, keras-applications
Required-by: 


In [3]:
## Imports libs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive_link = '/content/drive/My Drive/'
my_link = drive_link+'SKRIPSI/'

In [6]:
DATA_DIR = my_link+'dataset/'

# load repo with data if it is not exists
if not os.path.exists(DATA_DIR):
  print('Dataset not exist...')
else:
  print('Dataset Exist...')

Dataset Exist...


### Data Generator
###### ``Data Generator adalah kelas generator dataset yang dibungkus per batch sesuai dengan ukuran batch, ``
###### ``misal (batch_size, pixel_width, pixel_height, channel_size) = (8,128,128,3)``

In [0]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=256, augmentation=None, preprocessing=None): 
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.on_epoch_end()
        self.augmentation = augmentation
        self.preprocessing = preprocessing
        
    def __load__(self,id_name):
        ## Path
        image_path = os.path.join(self.path, "images", id_name) + ".png"
        mask_path = os.path.join(self.path, "masks", id_name) + ".png"
        
        ## Reading Image
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_size, self.image_size))
        
        ## Reading Masks
        mask = cv2.imread(mask_path, -1)
        mask = cv2.resize(mask, (self.image_size, self.image_size))
        mask = np.expand_dims(mask, axis=-1)
            
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        ## Jumlah Batch Terakhir Disesuaikan Sisa Data Yang Ada
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        ## Load Data Dengan Fungsi __load__
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)  
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)

        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [8]:
# Data jantung generate ids

train_path = DATA_DIR
jantung_path = '/content/drive/My Drive/SKRIPSI/dataset/jantung_idx.txt' # daftar nama data jantung ada disini
load_jt_ids = []

# Load idx dataset jantung
with open(jantung_path, 'rb') as handle: 
    load_jt_ids = [str(line.rstrip())[2:-1] for line in handle]

# Membagi dataset menjadi train, valid, test = 70%, 20%, 10% 
train_jt_ids = load_jt_ids[:int(5011*0.70)+1]
valid_jt_ids = load_jt_ids[int(5011*0.70)+1:-int(5011*0.10)]
test_jt_ids = load_jt_ids[-int(5011*0.10):]

print('Jumlah data jantung (train, valid, test) = (',len(train_jt_ids), ',', len(valid_jt_ids), ',', len(test_jt_ids),').')
print('Total dataset: ',len(train_jt_ids)+len(valid_jt_ids)+len(test_jt_ids))
print('Contoh data:',load_jt_ids[-1])
        

Jumlah data jantung (train, valid, test) = ( 3508 , 1002 , 501 ).
Total dataset:  5011
Contoh data: sol_033_z_pos_010_t_pos_020


### Hyperparameters

In [0]:
############____________________________________________________________############

# Definisikan model dan parameter yang diinginkan... 
# Definisi parameter ini akan digenerate otomatis oleh fungsi-fungsi dibawahnya

model_path = 'unet_mobilenet_jt_0.001_15_256_yes'

############____________________________________________________________############

Definisikan model_path: ``{decoder}_{encoder}_jt_{learning_rate}_{epochs}_{image_size}_{pretrained_imagenet?}``

**Berikut definisi encoder di sm-model:**
##### ``VGG``	: 'vgg16', 'vgg19'
##### ``ResNet``	: 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152'
##### ``SENet154``	: 	'senet154'
##### ``DenseNet``	: 	'densenet121', 'densenet169', 'densenet201'
##### ``Inception``	: 	'inceptionv3', 'inceptionresnetv2'
##### ``MobileNet``	: 	'mobilenet', 'mobilenetv2'
##### ``EfficientNet``	: 	'efficientnetb0', 'efficientnetb1', 'efficientnetb2', 'efficientnetb3', 'efficientnetb4', 'efficientnetb5' efficientnetb6', efficientnetb7'

**Berikut definisi decoder di sm-model:**
- unet
- linknet
- pspnet
- fpn

#### Fungsi dan Tools

In [0]:
# Posisi direktori model
save_dir =  [
              '/content/drive/My Drive/SKRIPSI/models/',
            ]

In [0]:
# Fungsi untuk mengolah history
import json, codecs, pickle

def saveHist(path, history):
    with open(path, 'wb') as handle: # saving the history of the model
        pickle.dump(history, handle)

def loadHist(path):
    n = {} # set history to empty
    if os.path.exists(path): # reload history if it exists
        with open(path, 'rb') as handle: # loading old history 
            n = pickle.load(handle)
    return n

def appendHist(h1, h2):
    if h1 == {}:
        return h2
    else:
        dest = {}
        for key, value in h1.items():
            dest[key] = value + h2[key]
        return dest

In [0]:
import segmentation_models as sm

# Mengambil Model Yang Sudah Ada
def take_model(model_path):
    print('Melanjutkan: ',model_path)
    md_ = model_path.split('_')
    lr = float(md_[3])
    epochs = int(md_[4])
    image_size = int(md_[5])
    pretrained = md_[6]
    batch_size = 8
    print('lr:',lr,', epochs:',epochs,', im_size:',image_size,', prt: '+pretrained)
    # optim, loss, metric harus sesuai dengan model yang ditraining sebelumnya
    optim = keras.optimizers.Adam(lr)
    total_loss = sm.losses.JaccardLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), sm.metrics.Dice]

    model = keras.models.load_model(save_dir[0]+model_path+'/'+model_path+'.h5', custom_objects={'jaccard_loss': total_loss, 'iou_score':metrics[0], 'f1-score':metrics[1]})

    return model, lr, epochs, image_size, pretrained, batch_size, metrics, total_loss, optim

# Mengatur Dataset
def set_data(train_ids,test_ids,valid_ids):
    train_gen = DataGen(train_ids, train_path, image_size=image_size, batch_size=batch_size)
    test_gen = DataGen(test_ids, train_path, image_size=image_size, batch_size=batch_size)
    valid_gen = DataGen(valid_ids, train_path, image_size=image_size, batch_size=batch_size)

    train_steps = len(train_ids)//batch_size
    test_steps = len(test_ids)//batch_size
    valid_steps = len(valid_ids)//batch_size

    return train_gen,test_gen,valid_gen,train_steps,test_steps,valid_steps

# Melanjutkan Training
def cont_training(model):
    print('Lanjutkan Training... '+model.name)   
    path = save_dir[0]+model.name+'/'+model.name
    old_hist = loadHist(path+'_history.bin')
    initial_epoch = len(old_hist['loss'])
    print('Dari epochs',initial_epoch)
    if initial_epoch>=500 or initial_epoch==50 or initial_epoch==250:
        print("Batal Training, Model Sudah Memenuhi Epochs "+str(initial_epoch))
    else:
        # define callbacks for learning rate scheduling and best checkpoints saving
        callbacks = [
            keras.callbacks.ModelCheckpoint(path+'_best_weights.h5', monitor='val_iou_score', save_weights_only=True, save_best_only=True, mode='max'),
        ]
        saveHist(path+'_history_temp_'+str(initial_epoch)+'.bin', old_hist)
        model.save(path+'_temp_'+str(initial_epoch)+'.h5')
        model.fit_generator(train_gen, 
                            validation_data=valid_gen, 
                            steps_per_epoch=train_steps, 
                            validation_steps=valid_steps, 
                            epochs=initial_epoch+epochs,
                            initial_epoch=initial_epoch,
                            callbacks=callbacks)
        print('Menyimpan Model...')    
        model.save(path+'.h5')
        history = model.history
        model.history.history = appendHist(old_hist, history.history)
        saveHist(path+'_history.bin', model.history.history)
        if initial_epoch>=400:
            train_vis(model.history.history, path, doing="save")
        print('Berhasil Menyimpan Model')

# Membuat Model SM Baru
def take_sm(model_path):
    print('Memulai SM-Model: ',model_path)
    try:
        # apakah model sudah pernah dibuat?
        os.mkdir(save_dir[0]+model_path)
    except:
        print('Overwrite '+model_path)
    md_ = model_path.split('_')
    lr = float(md_[3])
    epochs = int(md_[4])
    image_size = int(md_[5])
    pretrained = md_[6]
    batch_size = 8
    print('lr:',lr,', epochs:',epochs,', im_size:',image_size,', prt: '+pretrained)
    # optimizer manual setting 
    optim = keras.optimizers.Adam(lr)
    # loss function manual setting
    total_loss = sm.losses.JaccardLoss()
    # metric evaluation manual setting
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]
    if pretrained=='yes': encoder_weights = 'imagenet'
    else: encoder_weights = None
    if md_[0] is 'unet':
        model = sm.Unet(md_[1], classes=1, activation='sigmoid', input_shape=(image_size,image_size,3), encoder_weights=encoder_weights) 
    elif md_[0] is 'linknet':
        model = sm.Linknet(md_[1], classes=1, activation='sigmoid', input_shape=(image_size,image_size,3), encoder_weights=encoder_weights) 
    elif md_[0] is 'pspnet':
        model = sm.PSPNet(md_[1], classes=1, activation='sigmoid', input_shape=(image_size,image_size,3), encoder_weights=encoder_weights) 
    elif md_[0] is 'fpn':
        model = sm.FPN(md_[1], classes=1, activation='sigmoid', input_shape=(image_size,image_size,3), encoder_weights=encoder_weights) 
    else: 
        model = sm.Unet(md_[1], classes=1, activation='sigmoid', input_shape=(image_size,image_size,3), encoder_weights=encoder_weights) 
    model.compile(optim, total_loss, metrics)
    # auto set model name with model_path
    model.name = model_path
    return model, lr, epochs, image_size, pretrained, batch_size, metrics, total_loss, optim


#### Persiapan Model

In [0]:
##########____________________________________________ Pengaturan Model dan Dataset ____________________________________________##########

### Untuk membuat model sm baru, menggunakan fungsi take_sm() ###
model, lr, epochs, image_size, pretrained, batch_size, metrics, total_loss, optim = take_sm(model_path)
train_gen, test_gen, valid_gen, train_steps, test_steps, valid_steps = set_data(train_jt_ids,test_jt_ids,valid_jt_ids)

#### Fungsi Training Testing dan Evaluasi

In [0]:
from matplotlib import pyplot as plt
from keras.utils import plot_model
import json, codecs, pickle
import pandas as pd

# helper function for training visualization
def train_vis(history, model_save_dir, doing="show"):
    # Plot training & validation iou_score values
    plt.figure(figsize=(30, 5))
    plt.subplot(121)
    plt.plot(history['iou_score'])
    plt.plot(history['val_iou_score'])
    plt.title('Model iou_score')
    plt.ylabel('iou_score')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(122)
    plt.plot(history['loss'])
    plt.plot(history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    if (doing=="save"):
        plt.savefig(model_save_dir+'_plot.png')
        print("Success Saving Plot")
        plt.clf()
    else: 
        plt.show()

# helper function for training 
def train_fit(model_, epochs=epochs, pretrain=False):
    print("Training for "+model_.name)
    callbacks = [
        keras.callbacks.ModelCheckpoint(save_dir[0]+"{}/{}".format(model_.name,model_.name)+'_best_weights.h5', monitor='val_iou_score', save_weights_only=True, save_best_only=True, mode='max'),
    ]
    model_.fit_generator( train_gen, 
                          validation_data=valid_gen, 
                          steps_per_epoch=train_steps, 
                          validation_steps=valid_steps, 
                          epochs=epochs,
                          callbacks=callbacks)
        
    if pretrain:
        return 0
        
    try:
        os.mkdir(save_dir[0]+model_.name)
    except FileExistsError:
        print('Directory not created, '+model_.name+' was exist!')

    model_save_dir = save_dir[0]+model_.name+'/'+model_.name

    plot_model(model_, show_shapes=True, to_file=model_save_dir+'_architecture.png')

    train_vis(model_.history.history, model_save_dir, doing="save")

    with open(model_save_dir+'_history.bin', 'wb') as handle:
        pickle.dump(model_.history.history, handle)
    
    model_.save(model_save_dir+'.h5')
    print("Success Saving Model")

# helper function for testing 
def test_eval(model_):
    i_=0
    list_of_test = []
    print("Testing for "+model_.name)
    scores = model_.evaluate_generator(test_gen)
    list_of_test.append("Loss: {:.5}".format(scores[0]))
    print(list_of_test[i_])
    for metric, value in zip(metrics, scores[1:]):
        i_ += 1
        list_of_test.append("mean {}: {:.5}".format(metric.__name__, value))
        print(list_of_test[i_])

    model_save_dir = save_dir[0]+model_.name+'/'+model_.name
    
    with open(model_save_dir+'_scores.txt', 'w') as f:
        for item in list_of_test:
            f.write("%s\n" % item)


# helper function for data visualization
def visualize(**images):
    """PLot images in one row."""
    n = len(images)
    plt.figure(figsize=(16, 5))
    for i, (name, image) in enumerate(images.items()):
        plt.subplot(1, n, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.title(' '.join(name.split('_')).title())
        plt.imshow(image)
    plt.show()
    
# helper function for data visualization    
def denormalize(x):
    """Scale image to range 0..1 for correct plot"""
    x_max = np.percentile(x, 98)
    x_min = np.percentile(x, 2)    
    x = (x - x_min) / (x_max - x_min)
    x = x.clip(0, 1)
    return x

# helper function for test result and visualization
def test_vis(model, test_ids=test_jt_ids, evals=False):
    # new definition test data 
    test_dataset = DataGen(test_ids, train_path, image_size=image_size, batch_size=1)
    
    loss_ = []
    ious_ = []
    f1s_ = []
    task_ = ''

    if evals:
        jml = len(test_dataset)
        ids = range(0,jml)
        task_ = 'Sukses mengevaluasi model '+model.name+', dengan jumlah data_test: '+str(jml)
    else:
        #n = 5
        #ids = np.random.choice(np.arange(len(test_dataset)), size=n)
        jml = len(test_dataset)
        ids = range(0,jml)

    # visualize
    for i in ids:
        image, gt_mask = test_dataset[i]
        image = np.expand_dims(image[0], axis=0)
        pr_mask = model.predict(image).round()
      
        if not evals:
            visualize(
                image=denormalize(image.squeeze()),
                gt_mask=gt_mask[0][..., 0].squeeze(),
                pr_mask=pr_mask[0][..., 0].squeeze(),
            )
        else:
            scores = model.evaluate(image,gt_mask)
            
            if (i+1) % 10 == 0 :
                print('['+str(i+1)+'/'+str(jml)+'] --- dari progres ---')
            
            loss_.append(scores[0])
            ious_.append(scores[1])
            f1s_.append(scores[2])

    if evals:
        df = pd.DataFrame({'id': test_ids[:jml], 'loss': loss_, 'iou-score': ious_, 'f1-score': f1s_})
        df.to_csv(save_dir[0]+model.name+'/'+model.name+'_eval_test.csv',index=False)
    print()
    print(task_)
  

In [0]:
round(0.1284*100,1)

In [0]:
# Load weights in the best scores for each overall epoh
# model.load_weights('/content/drive/My Drive/PENELITIAN/models/unet_mobilenet_17a_0.001_100_256_yes/unet_mobilenet_17a_0.001_100_256_yes_best_weights.h5')

#### Train The Models

In [0]:
## Melanjutkan Training
##cont_training(model)

# Training Model Baru
train_fit(model)

In [0]:
model = keras.models.load_model('/content/drive/My Drive/SKRIPSI/models/unet_mobilenet_jt_0.001_15_256_yes/unet_mobilenet_jt_0.001_15_256_yes.h5',custom_objects={'jaccard_loss': total_loss, 'iou_score':metrics[0], 'f1-score':metrics[1]})

In [0]:
model.load_weights('/content/drive/My Drive/SKRIPSI/models/unet_mobilenet_jt_0.001_15_256_yes/unet_mobilenet_jt_0.001_15_256_yes.h5')

#### Test The Models 
`*(Tidak perlu dijalankan jika masih checkpoint)*`

In [0]:
test_eval(model)

In [0]:
test_vis(model, test_ids=test_jt_ids, evals=True)


#### Evaluasi Gambar
``Print kondisi setiap channel pada setiap layer untuk gambar yang akan didefinisikan``

In [0]:
load_csvs = save_dir[0]+model.name+"/"+model.name+"_eval_test.csv"

In [0]:
pd.read_csv(load_csvs)

In [0]:
dfcsv = pd.read_csv(load_csvs).sort_values('iou-score')
nilai_iou = []
# dfcsv
for niou in dfcsv['iou-score']:
     nilai_iou.append(int(round(niou*100,0)))
nilai_iou[0] = 1


In [0]:
index_ = list(pd.DataFrame(nilai_iou)[0].value_counts().index)
values_ = list(pd.DataFrame(nilai_iou)[0].value_counts())

In [0]:
names = index_
values = values_

plt.figure(figsize=(14, 5))

plt.bar(names, values)
plt.xticks(list(range(0,101,2)))
plt.yticks(list(range(0,31,2)))
plt.xlabel('Skor IoU (Pembulatan 0 angka di belakang koma)')
plt.ylabel('Frekuensi')
plt.grid()
plt.suptitle('Sebaran Skor IoU pada 600 Data Uji')
plt.show()

In [0]:
names = index_
values = values_

plt.figure(figsize=(3, 15))

plt.boxplot(nilai_iou)
plt.ylabel('Skor IoU (%)')
plt.yticks(list(range(0,101,2)))
plt.grid()
plt.suptitle('Sebaran Skor IoU pada 600 Data Uji')
plt.show()

In [0]:
dfcsv = pd.read_csv(load_csvs).sort_values('iou-score')
print(dfcsv[dfcsv['iou-score']<0.1].count()[0])
print(dfcsv[dfcsv['iou-score']>=0.1][dfcsv['iou-score']<=0.5].count()[0])
print(dfcsv[dfcsv['iou-score']>0.5][dfcsv['iou-score']<=0.85].count()[0])
print(dfcsv[dfcsv['iou-score']>0.85][dfcsv['iou-score']<=0.95].count()[0])
print(dfcsv[dfcsv['iou-score']>0.95].count()[0])

In [0]:
pd.read_csv(load_csvs)['iou-score'].sum()/600

In [0]:
print('Hasil ini digunakan pada model yang dilatih dengan batch_size 8')
print()
test_dataset = DataGen(test_jt_ids, train_path, image_size=image_size, batch_size=8)
hasils = model.evaluate_generator(test_dataset)
print("Hasil pengujian dengan batch_size = 8 (sesuai ukuran pelatihan)")
print("Loss: ",hasils[0])
print("IoU-Score: ",hasils[1])
print("F1-Score: ",hasils[2])
print()

test_dataset = DataGen(test_jt_ids, train_path, image_size=image_size, batch_size=1)
hasils = model.evaluate_generator(test_dataset)
print("Hasil pengujian dengan batch_size = 1 (satu-per-satu gambar)")
print("Loss: ",hasils[0])
print("IoU-Score: ",hasils[1])
print("F1-Score: ",hasils[2])

In [0]:
dfcsv = pd.read_csv(load_csvs).sort_values('iou-score')
test_id1 = list(dfcsv['id'][dfcsv['iou-score']<0.1])
test_id2 = list(dfcsv['id'][dfcsv['iou-score']>=0.1][dfcsv['iou-score']<=0.5])
test_id3 = list(dfcsv['id'][dfcsv['iou-score']>0.5][dfcsv['iou-score']<=0.85])
test_id4 = list(dfcsv['id'][dfcsv['iou-score']>0.85][dfcsv['iou-score']<=0.95])
test_id5 = list(dfcsv['id'][dfcsv['iou-score']>0.95])

In [0]:
def fungsi_vis(i,test_id1):
  test_dataset = DataGen(test_id1, train_path, image_size=image_size, batch_size=1)
  plt.figure(figsize=(16, 5))
  image, gt_mask = test_dataset[i]
  image = np.expand_dims(image[0], axis=0)
  pr_mask = model.predict(image).round()
  scores = model.evaluate(image,gt_mask)
  b,g,r = cv2.split(image[0])
  rgb_img = cv2.merge([r,g,b])
  # imagess = np.expand_dims(rgb_img, axis=0)

  contours1, hierarchy1 = cv2.findContours(np.array(gt_mask[0], dtype=np.uint8).squeeze(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
  largest_contour1 = []
  largest_area = 0
  for contour in contours1:
      area = cv2.contourArea(contour)
      if area > largest_area:
          largest_area = area
          largest_contour1 = contour
  cv2.drawContours(rgb_img, [largest_contour1], -2, (1, 1, 0), 1)

  contours2, hierarchy2 = cv2.findContours(np.array(pr_mask, dtype=np.uint8).squeeze(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE) 
  largest_contour2 = []
  largest_area = 0
  for contour in contours2:
      area = cv2.contourArea(contour)
      if area > largest_area:
          largest_area = area
          largest_contour2 = contour
  cv2.drawContours(rgb_img, [largest_contour2], -1, (0, 1, 1), 1)
  ious = str(scores[1]*100)[:str(scores[1]*100).find(".")+3] + '%'
  titles = test_id1[i]+" ("+ious+")"
  plt.title(titles)
  plt.imshow(denormalize(rgb_img.squeeze()))

In [0]:
for i in range(len(['sol_002_z_pos_011_t_pos_006','sol_001_z_pos_002_t_pos_001','sol_001_z_pos_002_t_pos_001'])):
  fungsi_vis(i,['sol_002_z_pos_011_t_pos_006','sol_001_z_pos_002_t_pos_001','sol_001_z_pos_002_t_pos_001'])

In [0]:
for i in range(len(test_id3[50:100])):
  fungsi_vis(i,test_id3[50:100])

In [0]:
for i in range(len(test_id3[-70:-20])):
  fungsi_vis(i,test_id3[-70:-20])

In [0]:
for i in range(len(test_id4[:20])):
  fungsi_vis(i,test_id4[:20])

In [0]:
for i in range(len(test_id4[-120:-80])):
  fungsi_vis(i,test_id4[-120:-80])

In [0]:
for i in range(len(test_id5)):
  fungsi_vis(i,test_id5)

In [0]:
##########____________________________________________________________________________##########

# Gambar yang akan dievaluasi (Definisikan!!!)
jenisimg = ['own','97','90','85','70','50','20','0,01','0,001'] # Kategori Berdasarkan Mean-IoU-Scores
eval_ids = ['sol_001_z_pos_002_t_pos_001','sol_001_z_pos_002_t_pos_015','sol_001_z_pos_002_t_pos_016','sol_001_z_pos_002_t_pos_017','sol_001_z_pos_002_t_pos_019','sol_001_z_pos_002_t_pos_020','sol_001_z_pos_003_t_pos_002','sol_001_z_pos_003_t_pos_003','sol_001_z_pos_003_t_pos_004']

link_fldr = '/content/drive/My Drive/SKRIPSI/visualisasi/'

##########____________________________________________________________________________##########

In [0]:
eval_data = DataGen(eval_ids, train_path, image_size=image_size, batch_size=1)
x,y = eval_data.__getitem__(0)

In [0]:
from keras import backend as K

# Print Image Input Layers
def printInput(x,y,i):
    print(x[0].shape)
    b,g,r = cv2.split(x[0])
    rgb_img = cv2.merge([r,g,b]) 
    pr = model.predict(x)
    fig, axs = plt.subplots(1, 6, figsize=(30,5))
    fig.suptitle('(0) Layer Input')
    axs[0].imshow(rgb_img) # Image
    axs[0].set_title('Image '+eval_ids[i]+' with score: '+jenisimg[i]+'%')
    axs[1].imshow(x[0][:,:,0]) # Blue
    axs[1].set_title('Blue channel '+eval_ids[i]+' with score: '+jenisimg[i]+'%')
    axs[2].imshow(x[0][:,:,1]) # Green
    axs[2].set_title('Green channel '+eval_ids[i]+' with score: '+jenisimg[i]+'%')
    axs[3].imshow(x[0][:,:,2]) # Red
    axs[3].set_title('Red channel '+eval_ids[i]+' with score: '+jenisimg[i]+'%')
    axs[4].imshow(y[0][:,:,0]) # Mask
    axs[4].set_title('Mask '+eval_ids[i]+' with score: '+jenisimg[i]+'%')
    axs[5].imshow(pr[0][:,:,0]) # Predict
    axs[5].set_title('Predict '+eval_ids[i]+' with score: '+jenisimg[i]+'%')

    fig.savefig(link_fldr+eval_ids[i]+'/(0) Layer Input.png')
    print("Success save layer input...")

# Pembagi Plot Size 
def rec(size_,pem=2,pen=2):
    hs = size_/4
    if size_ == 1: return pem-pem, pen-pen
    if hs == 1: return pem, pen
    elif hs % 4 != 0: return pem, pen*2
    else: return rec(hs,pem*2,pen*2)

# Print Image Every Layers
def printLayer(model,x,k,first_,last_):
    for i in range(first_,last_):
        print('('+str(i)+') Layer '+model.layers[i].name+'.png')
        if 'concat' in model.layers[i].name or 'pad' in model.layers[i].name:
            print('Skip layer '+model.layers[i].name)
        else:
            get_output = K.function([model.layers[0].input],
                                    [model.layers[i].output])
            
            layer_output = get_output([x])[0]

            pem, pen = rec(len(layer_output[0][0][0]))

            if pem == 0:
                plt.imshow(layer_output[0][...,0])
                plt.title("Ch_1")
                # Save Subplot.
                plt.savefig(link_fldr+eval_ids[k]+'/('+str(i)+') Layer '+model.layers[i].name+'.png')
            else:

                # Create a Subplot.
                fig, axs = plt.subplots(pen, pem, figsize=(32,32*pen//pem))
                fig.suptitle('('+str(i)+') Layer '+model.layers[i].name)
                
                for ii in range(pen):
                    for j in range(pem):
                        axs[ii, j].imshow(layer_output[0][...,(ii*pem)+j])
                        axs[ii, j].axis('off')
                        axs[ii, j].set_title("Ch_"+str((ii*pem)+j+1))
                
                # Save Subplot.
                fig.savefig(link_fldr+eval_ids[k]+'/('+str(i)+') Layer '+model.layers[i].name+'.png')

            print("Success save layer "+model.layers[i].name+"...")
            print()

In [0]:
# Definisikan lt berdasarkan urutan gambar (dibagi supaya ram tidak penuh)
lt = 0

print('Percobaan kurang:',len(eval_ids)-lt-1,'gambar')


In [0]:
# Execute 1/2 data
for i in range(lt,lt+1):
    print(eval_ids[i])
    try:
        os.mkdir(link_fldr+eval_ids[i]+'/')
    except FileExistsError:
        print('Directory not created, '+eval_ids[i]+' was exist!')
    x_,y_ = eval_data.__getitem__(i)
    printInput(x_,y_,i)
    printLayer(model,x_,i,1,len(model.layers)//2)
    print()


In [0]:
    # Execute 2/2 data
for i in range(lt,lt+1):
   print(eval_ids[i])
    try:
        os.mkdir(link_fldr+eval_ids[i]+'/')
    except FileExistsError:
        print('Directory not created, '+eval_ids[i]+' was exist!')
    x_,y_ = eval_data.__getitem__(i)
    printInput(x_,y_,i)
    printLayer(model,x_,i,len(model.layers)//2,len(model.layers))
    print()

#### Finish !

![alt text]Finish !... Semua File Akan Otomatis Replace

**Penting !...** *``Cara memastikan model sudah 500 epochs yaitu dengan melihat 

---

size pada file "history.bin", ketika sudah berukuran **"91KB"** artinya model tersebut sudah 500 epochs...*